In [1]:
# filepath = '/media/DataStore/breitsbw/data/rinex/greenland/qaq10220.12o'
# filepath = '/media/DataStore/breitsbw/data/rinex/greenland/scor0220.12o'
# filepath = '/media/DataStore/breitsbw/data/rinex/greenland/thu20220.12o'
filepath = '/media/DataStore/breitsbw/data/rinex/greenland/thu30220.12o'

GPSTK will read uncompressed RINEX files ('o' extension).

We read record-by-record.

In [2]:
import gpstk
import numpy
from numpy import zeros, arange, array, nan

In [3]:
header, records = gpstk.readRinexObs(filepath)

In [4]:
N = 0
for record in records:
    N += 1
print('There are {0} records'.format(N))

There are 86400 records


Given that there are `N` records in the file (should be 86400 for 1 Hz data rate), we should preallocate our observable arrays accordingly.

In [5]:
obs_descriptions = [o.description for o in header.obsTypeList]
print('Observations:\n\n' + '\n'.join(obs_descriptions))

Observations:

C/A-code pseudorange
L1 Carrier Phase
L2 Carrier Phase
Pcode L1 pseudorange
Pcode L2 pseudorange
Doppler Frequency L1
Doppler Frequency L2
Signal-to-Noise L1
Signal-to-Noise L2


In [6]:
# we need to pick names for these variables that will be valid MATLAB/Python variable names
alternate_names = {'C/A-code pseudorange': 'psr_l1',
                   'L1 Carrier Phase': 'phs_l1',
                   'L2 Carrier Phase': 'phs_l2',
                   'Pcode L1 pseudorange': 'psr_l1p',
                   'Pcode L2 pseudorange': 'psr_l2p',
                   'Doppler Frequency L1': 'dopp_l1',
                   'Doppler Frequency L2': 'dopp_l2',
                   'Signal-to-Noise L1': 'snr_l1',
                   'Signal-to-Noise L2': 'snr_l2'}

.

.


In [7]:
header, records = gpstk.readRinexObs(filepath)

In [44]:
data = {}
for name in alternate_names.values():
    data[name] = nan * zeros((32, N))  # just GPS satellites

In [45]:
data['rx_ecef'] = tuple(header.antennaPosition[i] for i in range(3))

In [46]:
i = 0  # record number, i.e. seconds from start of day (1Hz data)
for record in records:
    record_dict = record.obs.asdict()
    for sat_id in record_dict.keys():
        prn = sat_id.id
        sat_obs_dict = record_dict[sat_id].asdict()
        for key in sat_obs_dict.keys():
            var_name = alternate_names[key.description]
            data[var_name][prn - 1, i] = sat_obs_dict[key].data
    i += 1

In [47]:
outpath = '/media/DataStore/breitsbw/data/parsed/greenland_data_2012-01-22/'
outfilename = filepath.split('/')[-1][:-3] + 'dat'
outfilepath = outpath + outfilename
print(outfilepath)

/media/DataStore/breitsbw/data/parsed/greenland_data_2012-01-22/thu30220.dat


In [48]:
numpy.savez(outfilepath, **data)

.

.

.

In [16]:
record.numSvs

11L

In [21]:
d = record.obs.asdict()

In [36]:
for sat_id in d.keys():
    print(sat_id.id)
    obs = d[sat_id]
    obs_dict = obs.asdict()
    types = obs_dict.keys()
    for t in types:
        o = obs_dict[t]
        print(t.description, o.data)
    break

32
('C/A-code pseudorange', 23473284.929)
('Pcode L2 pseudorange', 23473288.317)
('Signal-to-Noise L2', 42.0)
('Doppler Frequency L1', 1975.206)
('Signal-to-Noise L1', 47.0)
('L2 Carrier Phase', -8768108.152)
('Doppler Frequency L2', 1539.131)
('L1 Carrier Phase', -11261282.892)
('Pcode L1 pseudorange', 23473282.699)


In [48]:
obs_list = [o.description for o in header.obsTypeList]
print('\n'.join(obs_list))

C/A-code pseudorange
L1 Carrier Phase
L2 Carrier Phase
Pcode L1 pseudorange
Pcode L2 pseudorange
Doppler Frequency L1
Doppler Frequency L2
Signal-to-Noise L1
Signal-to-Noise L2


In [28]:
d[gpstk.gpstk.SatID(30, gpstk.SatID.systemGPS)]

KeyError: <gpstk.gpstk.SatID; proxy of <Swig Object of type 'gpstk::SatID *' at 0x7f361c7d5bd0> >

.

.

.

.

In [48]:
obs_list = [o.description for o in header.obsTypeList]
print('\n'.join(obs_list))

C/A-code pseudorange
L1 Carrier Phase
L2 Carrier Phase
Pcode L1 pseudorange
Pcode L2 pseudorange
Doppler Frequency L1
Doppler Frequency L2
Signal-to-Noise L1
Signal-to-Noise L2


### header meta data

The header supposedly contains information about what signal observables are listed in the file.

In [ ]:
obstype = header.RegisteredRinexObsTypes[0]

In [ ]:
for obstype in header.RegisteredRinexObsTypes:
    print(obstype.C1depend, obstype.EPdepend, obstype.L1depend, obstype.L2depend,
          obstype.P1depend, obstype.P2depend, obstype.PSdepend, obstype.depend, obstype.description)

In [1]:
import gpstk